<a href="https://colab.research.google.com/github/siddadel/kalidas/blob/main/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Dec  4 17:48:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers --quiet
!git clone https://github.com/allenai/comet-atomic-2020
!pip install -r ./comet-atomic-2020/requirements.txt --quiet
!wget https://storage.googleapis.com/ai2-mosaic-public/projects/mosaic-kgs/comet-atomic_2020_BART.zip
!unzip comet-atomic_2020_BART.zip

     |████████████████████████████████| 3.1 MB 4.2 MB/s 
     |████████████████████████████████| 61 kB 607 kB/s 
     |████████████████████████████████| 3.3 MB 46.2 MB/s 
     |████████████████████████████████| 895 kB 60.2 MB/s 
     |████████████████████████████████| 596 kB 79.9 MB/s 
Cloning into 'comet-atomic-2020'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 184 (delta 56), reused 90 (delta 32), pack-reused 56
Receiving objects: 100% (184/184), 7.16 MiB | 6.28 MiB/s, done.
Resolving deltas: 100% (62/62), done.
     |████████████████████████████████| 43 kB 1.0 MB/s 
     |████████████████████████████████| 90 kB 4.6 MB/s 
     |████████████████████████████████| 313 kB 38.6 MB/s 
     |████████████████████████████████| 9.1 MB 74.7 MB/s 
     |████████████████████████████████| 379 kB 60.4 MB/s 
     |████████████████████████████████| 829 kB 73.5 MB/s 
     |█████████████████████

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
data_root = "/content/drive/MyDrive/ANLP21/"
output_dir = "/content/drive/MyDrive/ANLP21/train_concats/"
output_file = data_root+'concatenated.csv'

Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(1,r'./comet-atomic-2020/models/comet_atomic2020_bart')
from generation_example import Comet
print("model loading ...")
comet = Comet("comet-atomic_2020_BART")
comet.model.zero_grad()
print("model loaded")

model loading ...


In [ ]:
import pandas as pd
cf = pd.read_csv(data_root+'character_tour_manual_part1_split.csv')
cf = cf[cf.apply(lambda row: row['dev_train_test'] in ['test' , 'train' ,'dev'], axis=1)]
cf['film'] = cf['film'].apply(lambda film: film.replace(".txt.txt.txt.txt.txt.txt.txt",".txt"))
cf_list = cf['film'].to_list()

In [ ]:
import os
done_list = [file.replace(".csv", "") for file in os.listdir(output_dir)]

def film_name_condition(film_name):
  return film_name in cf_list and film_name not in done_list

In [ ]:
import string
inference_backup = pd.read_csv('/content/drive/MyDrive/ANLP21/inference_backup.csv')

backup_films = inference_backup['film'].to_list()
backup_items = inference_backup['item'].to_list()

def from_backup(film, item, relation):
  if(film in backup_films) and (item in backup_items) and (relation in inference_backup.columns):
    d = (inference_backup[inference_backup.apply(lambda row: (row['film']==film) and (row['item'] == item) and (row[relation]!=""), axis=1)])
    inferences =  d[relation].to_list()[0].strip('][').split(', ')
    return [i.translate(str.maketrans('', '', string.punctuation)).strip() for i in inferences]
  else:
    return None

In [ ]:
import pandas as pd
df = pd.read_csv(data_root+"database.csv")
df = df[df.apply(lambda row: film_name_condition(row['film']) and pd.notnull(row['character']), axis=1 )]

In [ ]:
df

,Unnamed: 0,film,scene,item_ids,item,character,protagonist
9,9,trainspotting.txt,2,6,"RENTON (V.O.) says, ""Choose life. Choose a job...",RENTON (V.O.),RENTON
15,15,trainspotting.txt,3,3,"RENTON (V.O.) says, ""Choose good health, low c...",RENTON (V.O.),RENTON
27,27,trainspotting.txt,4,12,"RENTON (V.O.) says, ""Choose leisure wear and m...",RENTON (V.O.),RENTON
33,33,trainspotting.txt,5,4,"RENTON (V.O.) says, ""I chose not to choose lif...",RENTON (V.O.),RENTON
34,34,trainspotting.txt,5,5,"SICK BOY says, ""Goldfinger's better than Dr. N...",SICK BOY,RENTON
...,...,...,...,...,...,...,...
895272,895272,rko_281.txt,76,3,"WELLES says, ""You know, all this nightmare we ...",WELLES,WELLES
895273,895273,rko_281.txt,76,4,"MANK says, ""Yeah, you're probably right.""",MANK,WELLES
895275,895275,rko_281.txt,76,6,"MANK says, ""I'll tell ya something, kid. When ...",MANK,WELLES
895278,895278,rko_281.txt,76,9,"WELLES says, ""Will burn. Burn up. Burn out. B...",WELLES,WELLES


In [ ]:
relations = [ "xAttr",  "MotivatedByGoal", "xReact"]

In [ ]:
def infer(item):
  # results = []
  # comet_run_needed = False
  # for relation in relations:
  #   result = from_backup(film, item, relation)
  #   if result is None:
  #     comet_run_needed = True
  #     break
  #   else:
  #     results.append(result)
  # if(comet_run_needed):
    results = []
    queries= ["{} {} [GEN]".format(item, relation) for relation in relations]
    results = comet.generate(queries, decode_method="beam", num_generate=5)
    
    return results

film_group = df.groupby('film')
df_concat = []
film_names = []
for film, group in film_group:
    print(film)
    #The following three ways were quicker ways but they don't work for some reason
    # if you want to work with these make sure to convert the results array in infer into json or string before returning
    # group['inference'] = comet.generate(["{} {} [GEN]".format(group['item'], relation) for relation in relations], decode_method="beam", num_generate=5)
    # group['inference'] = group['item'].apply((lambda x: infer(x)), axis=1)
    # group['inference'] = infer(group['item'])
    # for i in range(len(relations)):
    #   # group[str(relations[i])] = group['inference'].apply((lambda x: parse_inf(x, i)), axis=1)
    #   group[str(relations[i])] = parse_inf(group['inference'], i)
    for i in range(len(relations)):
      group[relations[i]] = ["" for i in range(len(group))]

    rel_inf = []
    for ind in group.index:
      item = group['item'][ind]
      results = infer(item)
      for i in range(len(relations)):
        group[relations[i]][ind] = results[i]
    
    df_concat.append(group)
    film_names.append(film)
    group.to_csv(output_dir+film+'.csv')
    # files.download(output_dir+film+'.csv')

its_a_wonderful_life.txt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


jane_eyre.txt
jaws.txt
jay_and_silent_bob_strike_back.txt
jerry_maguire.txt
jfk.txt
juno.txt
jurassic_park.txt
jurassic_park_2_the_lost_world.txt
kafka.txt
kids.txt
klute.txt
l.a._confidential.txt
lethal_weapon.txt
life_as_a_house.txt
logans_run.txt
love_and_basketball.txt
major_league.txt
mash.txt
memento.txt
men_in_black.txt
midnight_cowboy.txt
millers_crossing.txt
minority_report.txt
mission_impossible_ii.txt
mr._smith_goes_to_washington.txt
never_been_kissed.txt
next_friday.txt
officer_and_a_gentleman_an.txt
on_the_waterfront.txt
one_flew_over_the_cuckoos_nest.txt
pet_sematary.txt
pitch_black.txt
platoon.txt
pleasantville.txt
psycho.txt
pulp_fiction.txt
punch_drunk_love.txt
rebel_without_a_cause.txt
reservoir_dogs.txt
resident_evil__romero__unproduced.txt
rko_281.txt
rocky.txt
rush_hour.txt
rush_hour_2.txt
saving_private_ryan.txt
schindlers_list.txt
serial_mom.txt
sex_lies_and_videotape.txt
shampoo.txt
signs.txt
sling_blade.txt
smokey_and_the_bandit.txt
some_like_it_hot.txt
south_p

In [ ]:
df = pd.concat([pd.read_csv(os.path.join(output_dir, _)) for _ in os.listdir(fileDir) if _.endswith(".csv")])
df.to_csv(output_dir+'concatenated-all-characters.csv')
